In [2]:
# %matplotlib inline
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from esmpy.models import edxs
from esmpy.models.EDXS_function import print_concentrations_from_W
from esmpy.conf import DATASETS_PATH
from pathlib import Path
from esmpy.datasets import spim
import esmpy.tables_utils as tu
from esmpy.estimators import SmoothNMF

In [2]:
s = hs.load("/home/adrien/travail/esmpy/generated_datasets/FpBrgCaPv_N293_paper/sample_0.hspy")
s.change_dtype("float64")

In [3]:
s.metadata.Sample.thickness = 0.0

In [4]:
G = s.build_G(norm = True)
est = SmoothNMF( n_components = 3,tol=0.00001, max_iter = 100, G = G,hspy_comp = True)

s.decomposition(algorithm=est)

It 10 / 100: loss 0.119,  0.811 it/s
It 20 / 100: loss 0.119,  0.826 it/s
It 30 / 100: loss 0.119,  0.815 it/s
It 40 / 100: loss 0.119,  0.814 it/s
It 50 / 100: loss 0.118,  0.816 it/s
It 60 / 100: loss 0.118,  0.814 it/s
It 70 / 100: loss 0.118,  0.813 it/s
It 80 / 100: loss 0.118,  0.814 it/s
It 90 / 100: loss 0.118,  0.817 it/s
exits because of relative change < tol: 1.0275400417968269e-05
Stopped after 92 iterations in 1.0 minutes and 53.0 seconds.
Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SmoothNMF()
  output_dimension=None
  centre=None
scikit-learn estimator:
SmoothNMF()


In [11]:
x = np.linspace(s.axes_manager[2].offset,s.axes_manager[2].offset + s.axes_manager[2].scale*s.axes_manager[2].size, num = s.axes_manager[2].size)
plt.plot(x,G()[:,10])

In [17]:
s.metadata.Sample.elements

['Hf', 'Cu', 'Ca', 'O', 'Ti', 'Nd', 'Sm', 'Mg', 'Si', 'Al', 'Fe', 'Lu']

In [6]:
W = est.W_
H = est.H_
fe1 = (W[10]@H).reshape((128,128))
fe2 = (W[11]@H).reshape((128,128))



In [3]:
from esmpy.models.absorption_edxs import absorption_correction, absorption_coefficient
A = absorption_correction(0.7,thickness=0.1, density=3.0, toa=90,elements_dict = {"26" : 1.0})

In [8]:
energy = 0.7
toa = s.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle

def abs_map(l_map, energy, toa, mass_thickness, elements_dict = {"26" :1.0}) : 
    mu = absorption_coefficient(energy,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi = mu*mass_thickness/np.sin(rad_toa)
    return l_map*((1 - np.exp(-chi))/chi)


m = abs_map(fe1,energy,toa,np.random.rand(128,128))

plt.imshow(m)

In [28]:
# <examples/doc_parameters_basic.py>
import numpy as np

from lmfit import Minimizer, Parameters, report_fit
from tqdm import tqdm

# define objective function: returns the array to be minimized
def absorption_fit(params,data,energy1, energy2, toa, elements_dict = {}):
    """Model a decaying sine wave and subtract data."""
    mt = params['mass_thickness']
    mu1 = absorption_coefficient(energy1,atomic_fraction=False,elements_dict= elements_dict)
    mu2 = absorption_coefficient(energy2,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi1 = mu1*mt/np.sin(rad_toa)
    chi2 = mu2*mt/np.sin(rad_toa)
    model = (1 - np.exp(-chi2))/(1 - np.exp(-chi1))*(mu1/mu2)
    return model - data


# create a set of Parameters
params = Parameters()
params.add('mass_thickness', value=0.0085, min=1e-10)

# do fit, here with the default leastsq algorithm
mt_map = np.zeros_like(s.data[:,:,0])
data = fe1/fe2
for i in tqdm(range(mt_map.shape[0])) : 
    for j in range(mt_map.shape[1]) :
        curr_data = data[i,j]
        minner = Minimizer(absorption_fit, params, fcn_args=(curr_data,0.7,6.4,s.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, {'26' : 1.0} ))
        result = minner.minimize()
        mt_map[i,j] = result.params['mass_thickness'].value
        params['mass_thickness'].value = result.params['mass_thickness'].value

In [2]:
def absorption_mass_thickness(x,mass_thickness, toa = 90, atomic_fraction = True, *, elements_dict = {"Si" : 1.0}) : 
    mu = absorption_coefficient(x,atomic_fraction,elements_dict = elements_dict)
    rad_toa = np.deg2rad(toa)
    if type(mu) == float : 
        chi = mu*mass_thickness/np.sin(rad_toa)
    else : 
        chi = mu[:,np.newaxis]@((mass_thickness.reshape(-1)[:,np.newaxis]).T)/np.sin(rad_toa)
    return (1-np.exp(-chi))/chi

In [66]:
ed

{'Hf': 1.4146027014913096,
 'Cu': 5.808172796043127,
 'Ca': 4.286308983161415,
 'O': 17.10360084418461,
 'Ti': 0.0524235259576504,
 'Nd': 0.0993262299689619,
 'Sm': 0.11049384289768606,
 'Mg': 14.81147665353569,
 'Si': 11.079956621707547,
 'Al': 1.136738683481501,
 'Fe': 2.505167703634514,
 'Lu': 0.19468596910149402}

In [81]:
plt.plot(s.data.sum(axis=(0,1)))

In [41]:
ed = {}
i = 0
for elt in s.metadata.Sample.elements : 
    if elt == "Fe" : 
        i += 1
    ed[elt] = np.mean(est.W_[i])
    i+=1

In [47]:
absc = absorption_mass_thickness(x,mt_map,toa = s.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, elements_dict = ed)

In [48]:
abscr = absc.T.reshape((128,128,1980))

In [49]:
sabscr = hs.signals.Signal1D(abscr)

In [34]:
sabscr.plot()

In [50]:
s.data = s.data/sabscr.data

In [52]:
sabscr.plot()

In [56]:
G = s.build_G(norm = True)
est = SmoothNMF( n_components = 3,tol=0.00001, max_iter = 100, G = G,hspy_comp = True)

s.decomposition(algorithm=est)

It 10 / 100: loss 0.137,  0.829 it/s
It 20 / 100: loss 0.136,  0.819 it/s
It 30 / 100: loss 0.136,  0.820 it/s
It 40 / 100: loss 0.136,  0.813 it/s
It 50 / 100: loss 0.135,  0.815 it/s
It 60 / 100: loss 0.135,  0.818 it/s
It 70 / 100: loss 0.135,  0.818 it/s
exits because of relative change < tol: 1.0231198032930053e-05
Stopped after 73 iterations in 1.0 minutes and 29.0 seconds.
Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SmoothNMF()
  output_dimension=None
  centre=None
scikit-learn estimator:
SmoothNMF()


In [57]:
s.plot_decomposition_results()

In [31]:
spim = hs.load("eds_rebin.hspy")

spim.set_signal_type("EDS_ESMPY")
spim.set_additional_parameters(thickness = 0.0,density = 2.1,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")
spim.metadata.Sample.elements = ['Al','C','Cr','Cu','F','Fe','Mg','N','Na','Ni','O','S','Si']

# spim.set_analysis_parameters(beam_energy = 300,
# azimuth_angle = 0.0,
# elevation_angle = 35.0,
# elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti', 'Zr'],
# tilt_stage = -6.071,
# thickness = 80e-7,
# density = 2.1,
# detector_type = "Det_eff.txt",
# width_slope = 0.01,
# width_intercept = 0.065,
# xray_db = "300keV_corentin_xrays.json")

cropped_spim = spim.inav[1:195,:].isig[:7.8]

/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/hyperspy/misc/utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/hyperspy/io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


<AxesSubplot:xlabel='X-ray energy (keV)', ylabel='Intensity'>

In [9]:
spectrum = hs.load("phyllo_spectre_moyen.hspy")
spectrum.set_signal_type("EDS_ESMPY")
spectrum.axes_manager[0].offset += +0.000
spectrum.set_additional_parameters(thickness = 0.0,density = 2.1,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")
spectrum.metadata.Sample.elements = ['K','Pt','Al','C','Cr','Cu','F','Fe','Ga','Mg','N','Na','Ni','O','S','Si']

cropped_spim = spectrum.isig[:7.8]

G_ref = cropped_spim.build_G("bremsstrahlung", norm = True, reference_elt={'26' : 3.0,"24" : 3.0})

est_ref = SmoothNMF(n_components = 1, tol = 0.000001, max_iter = 10000000, G = G_ref)
 
est_ref.fit(cropped_spim.data[:,np.newaxis])

fW = est_ref.W_
fG = est_ref.G_

x = np.linspace(cropped_spim.axes_manager[0].offset, cropped_spim.axes_manager[0].offset + cropped_spim.axes_manager[0].scale*cropped_spim.axes_manager[0].size, num = cropped_spim.axes_manager[0].size)

# <examples/doc_parameters_basic.py>
import numpy as np

from lmfit import Minimizer, Parameters, report_fit
from tqdm import tqdm
from esmpy.models.absorption_edxs import absorption_correction, absorption_coefficient

# define objective function: returns the array to be minimized
def absorption_fit(params,data,energy1, energy2, toa, elements_dict = {}):
    """Model a decaying sine wave and subtract data."""
    mt = params['mass_thickness']
    mu1 = absorption_coefficient(energy1,atomic_fraction=False,elements_dict= elements_dict)
    mu2 = absorption_coefficient(energy2,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi1 = mu1*mt/np.sin(rad_toa)
    chi2 = mu2*mt/np.sin(rad_toa)
    model = (1 - np.exp(-chi1))/(1 - np.exp(-chi2))*(mu2/mu1)
    return model - data


# create a set of Parameters
params = Parameters()
params.add('mass_thickness', value=0.0005, min=1e-10)


cr_ratio = fW[2]/fW[3]
fe_ratio = fW[6]/fW[7]
# do fit, here with the default leastsq algorithm


curr_data = fe_ratio
minner = Minimizer(absorption_fit, params, fcn_args=(curr_data,0.7,6.4,cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, {"26" : 1.0} ))
result = minner.minimize()
print(result.params['mass_thickness'].value)
        # params['mass_thickness'].value = result.params['mass_thickness'].value

plt.plot(x,cropped_spim.data,'ko',markersize = 3.5,label = "Experimental")
plt.plot(x,fG@fW,'r',linewidth = 2, label = "Simulated")
plt.legend()
plt.xlabel("Energy (keV)")
plt.ylabel("Intensity (cts)")
plt.ylim(0,4)
plt.tight_layout()

It 10 / 10000000: loss 0.018,  22.307 it/s
It 20 / 10000000: loss 0.017,  23.018 it/s
It 30 / 10000000: loss 0.017,  23.141 it/s
It 40 / 10000000: loss 0.017,  23.591 it/s
exits because of relative change < tol: 2.8233664899750605e-06
Stopped after 46 iterations in 0.0 minutes and 2.0 seconds.
0.13044578860631506


In [10]:
spectrum = hs.load("phyllo_spectre_moyen.hspy")
spectrum.set_signal_type("EDS_ESMPY")
spectrum.axes_manager[0].offset += +0.000
spectrum.set_additional_parameters(thickness = 0.0004934,density = 1.0,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")
spectrum.metadata.Sample.elements = ['Al','C','Cr','Cu','F','Fe','Ga','Mg','N','Na','Ni','O','S','Si']

cropped_spim = spectrum.isig[:7.8]

G_nref = cropped_spim.build_G("bremsstrahlung", norm = True, reference_elt={})

est_nref = SmoothNMF(n_components = 1, tol = 0.000001, max_iter = 10000000, G = G_nref)
 
est_nref.fit(cropped_spim.data[:,np.newaxis])

fWn = est_nref.W_
fGn = est_nref.G_

exit because of negative decrease -1.0758475957328706e-07: 0.060554917988197836, 0.06055502557295741
Stopped after 9 iterations in 0.0 minutes and 0.0 seconds.


In [29]:
W = np.vstack((fW[:2], fW[3:6],fW[7:]))
nW = W/W.sum(axis = 0,keepdims = True)
siW = nW/nW[-3]


i = 0
for j in cropped_spim.metadata.Sample.elements : 
    if j == 'Cu' : 
        pass
    else : 
        print("{} : {}".format(j,siW[i]))
        i +=1

Al : [0.08233703]
C : [0.17744989]
Cr : [0.01102022]
F : [0.00979825]
Fe : [0.37928018]
Ga : [0.24456582]
Mg : [0.02210331]
N : [0.87711577]
Na : [0.01990752]
Ni : [0.04110368]
O : [0.01276253]
S : [1.73782993]
Si : [0.0381382]


In [33]:
fW/fW[-3]

array([[0.08233703],
       [0.17744989],
       [0.        ],
       [0.01102022],
       [0.00979825],
       [0.37928018],
       [0.08315487],
       [0.24456582],
       [0.02210331],
       [0.87711577],
       [0.01990752],
       [0.04110368],
       [0.01276253],
       [1.73782993],
       [0.0381382 ],
       [1.        ],
       [0.        ],
       [0.13364745]])

In [11]:
fWn/fWn[-3]

array([[0.09557353],
       [0.25249888],
       [0.        ],
       [0.0325192 ],
       [0.35123119],
       [0.27866166],
       [0.0298309 ],
       [0.88454371],
       [0.06314531],
       [0.0593343 ],
       [0.01575562],
       [1.7710745 ],
       [0.04194237],
       [1.        ],
       [0.        ],
       [0.05651161]])

In [35]:
G_no_ref = cropped_spim.build_G("bremsstrahlung",norm = True, reference_elt = {})
est_no_ref = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 1000000, G = G_no_ref,hspy_comp = True)

cropped_spim.decomposition(algorithm=est_no_ref)

AttributeError: It is not possible to decompose a dataset with navigation_size < 2

In [153]:
cropped_spim.save("eds_rebin_results_no_thickness_noCu.hspy")

ERROR:hyperspy.io_plugins._hierarchical:The writer could not write the following information in the file: decomposition_algorithm : SmoothNMF()
Traceback (most recent call last):
  File "/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/hyperspy/io_plugins/_hierarchical.py", line 786, in dict2group
    group.attrs[key] = value
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/h5py/_hl/attrs.py", line 103, in __setitem__
    self.create(name, data=value)
  File "/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/h5py/_hl/attrs.py", line 180, in create
    htype = h5t.py_create(original_dtype, logical=True)
  File "h5py/h5t.pyx", line 1663, in h5py.h5t.py_create
  File "h5py/h5t.pyx", line 1687, in h5py.h5t.py_create
  File "h5py/h5t.pyx", line 1747, in h5py.h5t.py_create
TypeError: Object dtype dtype

In [5]:
G_ref = cropped_spim.build_G("bremsstrahlung",norm = True, reference_elt = {"26":3.0})
est_ref = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 1000000, G = G_ref,hspy_comp = True)

cropped_spim.decomposition(algorithm=est_ref)

It 10 / 1000000: loss 1.189,  1.064 it/s
It 20 / 1000000: loss 1.080,  1.087 it/s
It 30 / 1000000: loss 1.022,  1.096 it/s


/home/adrien/travail/esmpy/esmpy/updates.py:49: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(num / (new_x), axis=0) - 1


It 40 / 1000000: loss 0.988,  1.101 it/s
It 50 / 1000000: loss 0.970,  1.103 it/s
It 60 / 1000000: loss 0.960,  1.105 it/s
It 70 / 1000000: loss 0.953,  1.108 it/s
It 80 / 1000000: loss 0.948,  1.109 it/s
It 90 / 1000000: loss 0.945,  1.110 it/s
It 100 / 1000000: loss 0.942,  1.110 it/s
It 110 / 1000000: loss 0.939,  1.111 it/s
It 120 / 1000000: loss 0.936,  1.111 it/s
It 130 / 1000000: loss 0.933,  1.111 it/s
It 140 / 1000000: loss 0.930,  1.111 it/s
It 150 / 1000000: loss 0.927,  1.109 it/s
It 160 / 1000000: loss 0.924,  1.108 it/s
It 170 / 1000000: loss 0.922,  1.106 it/s
It 180 / 1000000: loss 0.920,  1.104 it/s
It 190 / 1000000: loss 0.918,  1.102 it/s
It 200 / 1000000: loss 0.916,  1.099 it/s
It 210 / 1000000: loss 0.914,  1.097 it/s
It 220 / 1000000: loss 0.912,  1.096 it/s
Dicotomy stopped for maximum number of iterations with an error of : 6.995809108611972e-05
Dicotomy stopped for maximum number of iterations with an error of : 0.0009014811601451012
It 230 / 1000000: loss 0.9

In [7]:
W = est_ref.W_
H = est_ref.H_

iron_ratio = ((W[5]@H).reshape((68,194)))/((W[6]@H).reshape((68,194)))

# iron_ratio = (W[6]@H).reshape((68,195))/(W[7]@H).reshape((68,195)) 

In [10]:
ed = {}
i = 0
W = est_ref.W_
for elt in cropped_spim.metadata.Sample.elements : 
    if elt == "Fe" : 
        i += 1
    ed[elt] = np.mean(W[i])
    i+=1

ed

{'Al': 26.55877650574517,
 'C': 170.98798818647705,
 'Cr': 111.00852515399616,
 'Cu': 14.120609432997057,
 'F': 105.20790478235482,
 'Fe': 3483.35174164706,
 'Mg': 283.10872939836156,
 'N': 4.456662348101455,
 'Na': 33.596044364731306,
 'Ni': 437.53200464042936,
 'O': 934.9607960558623,
 'S': 860.0730185409587,
 'Si': 400.69007459615005}

In [11]:
# <examples/doc_parameters_basic.py>
import numpy as np

from lmfit import Minimizer, Parameters, report_fit
from tqdm import tqdm
from esmpy.models.absorption_edxs import absorption_correction, absorption_coefficient

# define objective function: returns the array to be minimized
def absorption_fit(params,data,energy1, energy2, toa, elements_dict = {}):
    """Model a decaying sine wave and subtract data."""
    mt = params['mass_thickness']
    mu1 = absorption_coefficient(energy1,atomic_fraction=False,elements_dict= elements_dict)
    mu2 = absorption_coefficient(energy2,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi1 = mu1*mt/np.sin(rad_toa)
    chi2 = mu2*mt/np.sin(rad_toa)
    model = (1 - np.exp(-chi1))/(1 - np.exp(-chi2))*(mu2/mu1)
    return model - data


# create a set of Parameters
params = Parameters()
params.add('mass_thickness', value=0.00085, min=1e-10)

# do fit, here with the default leastsq algorithm
mt_map = np.zeros_like(cropped_spim.data[:,:,0])
for i in tqdm(range(mt_map.shape[0])) : 
    for j in range(mt_map.shape[1]) :
        curr_data = iron_ratio[i,j]
        minner = Minimizer(absorption_fit, params, fcn_args=(curr_data,0.7,6.4,cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, {"26" : 1.0} ))
        result = minner.minimize()
        mt_map[i,j] = result.params['mass_thickness'].value
        # params['mass_thickness'].value = result.params['mass_thickness'].value

100%|██████████| 68/68 [02:17<00:00,  2.03s/it]


In [12]:
plt.imshow(mt_map)

In [13]:
offset = cropped_spim.axes_manager[2].offset
size = cropped_spim.axes_manager[2].size
scale = cropped_spim.axes_manager[2].scale
x = np.linspace(offset, scale*size + offset, num = size)
absc = absorption_mass_thickness(x,mt_map,toa = cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, elements_dict = {"F" : 0.47,"Fe" : 0.25, "Si" : 1.0, "O" : 2.6, "Mg" : 0.9, "C" : 0.6})
abscr = absc.T.reshape((cropped_spim.axes_manager[1].size,cropped_spim.axes_manager[0].size,size))

In [24]:
hs.signals.Signal1D(abscr).plot()

In [113]:
plt.plot(x,abs_corr.get_decomposition_model().mean().data, label = 'abs corr')
plt.plot(x,cropped_spim.mean().data,'ko', markersize = 1.3,  label = "exp")
plt.plot(x, no_abs.get_decomposition_model().mean().data, label = 'no thickness')
plt.plot(x,abs.get_decomposition_model().mean().data,label = "abs")
plt.legend()

In [14]:
ns = cropped_spim.copy()
ns.data = cropped_spim.data/abscr

In [196]:
cropped_spim.plot()

In [15]:
G_aft = ns.build_G("bremsstrahlung",norm = True, reference_elt={})
est_aft = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 1000000, G = G_aft,hspy_comp = True)

ns.decomposition(algorithm=est_aft)

It 10 / 1000000: loss 19.441,  1.057 it/s
It 20 / 1000000: loss 17.872,  1.086 it/s
It 30 / 1000000: loss 17.229,  1.097 it/s
It 40 / 1000000: loss 16.956,  1.097 it/s
It 50 / 1000000: loss 16.791,  1.100 it/s


/home/adrien/travail/esmpy/esmpy/updates.py:49: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(num / (new_x), axis=0) - 1


It 60 / 1000000: loss 16.706,  1.101 it/s
It 70 / 1000000: loss 16.658,  1.103 it/s
It 80 / 1000000: loss 16.629,  1.103 it/s
It 90 / 1000000: loss 16.609,  1.105 it/s
It 100 / 1000000: loss 16.596,  1.105 it/s
It 110 / 1000000: loss 16.586,  1.106 it/s
It 120 / 1000000: loss 16.578,  1.106 it/s
It 130 / 1000000: loss 16.572,  1.107 it/s
It 140 / 1000000: loss 16.566,  1.106 it/s
It 150 / 1000000: loss 16.561,  1.108 it/s
It 160 / 1000000: loss 16.557,  1.109 it/s
It 170 / 1000000: loss 16.554,  1.108 it/s
It 180 / 1000000: loss 16.550,  1.103 it/s
exits because of relative change < tol: 1.830164761111593e-05
Stopped after 188 iterations in 2.0 minutes and 51.0 seconds.
Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SmoothNMF()
  output_dimension=None
  centre=None
scikit-learn estimator:
SmoothNMF()


In [185]:
ns.save("eds_rebin_results_after_correction_noCu.hspy")

ERROR:hyperspy.io_plugins._hierarchical:The writer could not write the following information in the file: decomposition_algorithm : SmoothNMF()
Traceback (most recent call last):
  File "/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/hyperspy/io_plugins/_hierarchical.py", line 786, in dict2group
    group.attrs[key] = value
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/h5py/_hl/attrs.py", line 103, in __setitem__
    self.create(name, data=value)
  File "/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/h5py/_hl/attrs.py", line 180, in create
    htype = h5t.py_create(original_dtype, logical=True)
  File "h5py/h5t.pyx", line 1663, in h5py.h5t.py_create
  File "h5py/h5t.pyx", line 1687, in h5py.h5t.py_create
  File "h5py/h5t.pyx", line 1747, in h5py.h5t.py_create
TypeError: Object dtype dtype

In [53]:
W = np.vstack((est_aft.W_[:3], est_aft.W_[4:]))
nW = W/W.sum(axis = 0,keepdims = True)
sW = np.sum(nW,axis = 1)
siW = sW/sW[-3]

W1 = np.vstack((est_no_ref.W_[:5], est_no_ref.W_[6:]))
nW1 = W1/W1.sum(axis = 0,keepdims = True)
sW1 = np.sum(nW1,axis = 1)
siW1 = sW1/sW1[-5]

i = 0
for j in cropped_spim.metadata.Sample.elements : 
    if j == 'Cu' : 
        pass
    else : 
        print("{} : {}".format(j,siW[i]))
        i +=1

i = 0
for j in cropped_spim.metadata.Sample.elements : 
    if j == 'Cu' : 
        pass
    else : 
        print("{} : {}".format(j,siW1[i]))
        i +=1

In [20]:
ns.plot_decomposition_results()

In [29]:
cropped_spim.plot_decomposition_results()

In [21]:
factors = ns.get_decomposition_factors().data
plt.plot(factors[4])

In [22]:
H = est_aft.H_.reshape((7,68,194))
factors = est_ref.G_@est_ref.W_
for i,j in enumerate(H) : 
    new_mt = (mt_map*j).clip(min = 1e-10)
    abs_map = absorption_mass_thickness(x,new_mt,toa = cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, elements_dict = ed)
    print(np.isnan(factors[i]).sum())
    factors[i] /= np.mean(abs_map)

0
0
0
0
0
0
0


In [23]:
resu = (factors@est_ref.H_).sum(axis = 1 )/(194*68)

In [25]:
no_abs = hs.load("eds_rebin_results_no_thickness_noCu.hspy")

In [28]:
# plt.plot(x,abs.mean().data)
plt.plot(x+0.005,resu, label = 'abs_corr')
plt.plot(x,no_abs.get_decomposition_model().mean().data, label = "no")
plt.plot(x,cropped_spim.mean().data, 'ko', markersize = 1.5, label = "exp")
plt.legend()

In [132]:
np.isnan(factors).sum()

0

In [194]:
sp =  hs.load(filenames=["sp_C.hspy","sp_FeNiS.hspy","sp_FeO.hspy","sp_FeS.hspy","sp_Si_1.hspy","sp_Si_2.hspy","sp_ZnS.hspy"])
maps = hs.load("Map_mixing_model_7comp.hspy")

In [208]:
maps.plot(title = "truth")
ns.get_decomposition_loadings().plot()

In [212]:
fig, ax = plt.subplots()
ax.plot(sp[4]*10)
ax.plot(factors[:,0])

In [64]:
fig1,ax1 = plt.subplots()
ax1.plot(x,sp[1])
ax1.plot(x,factors[2])

In [60]:
fig2, ax2 = plt.subplots()
ax2.imshow(mt_map)

In [74]:
abs_corr.get_decomposition_loadings().plot(title = "Corrected")
abs.get_decomposition_loadings().plot(title = "absorption")
no_abs.get_decomposition_loadings().plot(title = "no thickness")

In [213]:
for i,j in enumerate(cropped_spim.metadata.Sample.elements) :
    print("element {}: {}".format(j,est_ref.W_[i])) 

element Al: [1.80653532e+01 1.32501425e+01 2.83217125e+01 9.91699934e+01
 5.68894549e-02 2.73717907e+00 7.17347118e+01]
element C: [9.04309064e-69 1.05949363e-55 1.93313691e+02 4.57029392e+00
 2.09194249e+00 8.07647914e+02 4.86555404e+02]
element Ca: [1.76161249e-31 3.84855744e-35 3.46870645e-43 7.07922023e-36
 0.00000000e+00 0.00000000e+00 5.24078420e-38]
element Cl: [21.36406601  4.51110665 32.76935764 13.13475589  0.38113707 15.22030853
 20.223993  ]
element Cr: [877.83612531   0.           0.           0.           0.
   0.           0.        ]
element Cu: [7.38172896e+01 2.44887186e+01 1.79932862e-13 3.47816548e+01
 0.00000000e+00 1.46827742e+01 1.67152932e+01]
element Fe: [1.74167705e+03 9.68662970e+02 4.55594428e+02 8.39185408e+01
 1.15513118e+00 4.69555041e+01 6.87361588e+01]
element Ga: [1.07546972e+04 4.45770281e+03 1.96922169e+02 1.38481752e+02
 1.50548815e+01 5.66318856e+00 6.39971689e+01]
element K: [4.20824318e-12 1.09951661e+01 4.41277670e-01 1.23487565e+02
 5.60015396e

In [69]:
no_abs = hs.load('eds_rebin_results_no_thickness.hspy')
abs = hs.load('eds_rebin_results_abs.hspy')
abs_corr = hs.load('eds_rebin_results_after_correction.hspy')

In [10]:
cropped_spim.plot_decomposition_results()

fW = est_aft.W_
print_concentrations_from_W(fW[:-2,:],elements = cropped_spim.metadata.Sample.elements)

Concentrations report
     p0     p1     p2     p3     p4     p5     p6
Al : 0.0000 0.0000 0.0000 0.0000 0.0073 0.0000 0.0039 
C  : 0.0268 0.2638 0.0381 0.6771 0.1590 0.7950 0.3157 
Ca : 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
Cl : 0.0000 0.0000 0.0000 0.0000 0.0004 0.0000 0.0002 
Cr : 0.1268 0.0000 0.0306 0.0000 0.0000 0.0000 0.0000 
Cu : 0.0106 0.0233 0.0054 0.0023 0.0065 0.0005 0.0031 
Fe : 0.4753 0.0000 0.2514 0.0059 0.0000 0.2014 0.0134 
Ga : 0.0000 0.0010 0.0000 0.0000 0.0118 0.0000 0.0022 
K  : 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
Mg : 0.0000 0.0396 0.0000 0.0247 0.0967 0.0000 0.0708 
Mn : 0.0531 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
N  : 0.0004 0.0030 0.0190 0.0088 0.0105 0.0003 0.0080 
Na : 0.0000 0.0000 0.0000 0.0029 0.0070 0.0000 0.0042 
Ni : 0.0018 0.2664 0.0000 0.0000 0.0000 0.0000 0.0000 
O  : 0.0135 0.3580 0.6545 0.2434 0.6155 0.0000 0.5107 
P  : 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
Pt : 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0

In [20]:
cfW = np.vstack((fW[:5],fW[6:]))
sfw = cfW.sum(axis = 1)
ssfw = sfw/sfw.sum()
nssfw = ssfw
for i,j in enumerate(ns.metadata.Sample.elements) : 
    print('{} : {}'.format(j,nssfw[i]))


Al : 0.0008433325704307077
C : 0.15424252985825934
Ca : 0.0
Cl : 4.964363423385926e-05
Cr : 0.05952107739002931
Cu : 0.25377404087608735
Fe : 0.0011676067052357627
Ga : 1.576935599265972e-269
K : 0.017277661911537304
Mg : 0.02221964101993724
Mn : 0.00683136763908825
N : 0.0012389001476346801
Na : 0.011470848470506985
Ni : 0.27293091547575665
O : 2.2771810881757245e-100
P : 1.3262284694353116e-125
Pt : 0.12211143193752587
S : 0.01784347391413898
Si : 7.183891900216825e-220
Ti : 6.885146069035893e-05
Zn : 0.0


In [112]:
abscr.shape

(68, 204, 1970)

In [115]:
plt.plot(np.mean(G()@est.W_@est.H_, axis = 1))
plt.plot(ns.mean())

In [104]:
ed = {}
i = 0
for elt in cropped_spim.metadata.Sample.elements : 
    if elt == "Fe" : 
        i += 1
    ed[elt] = np.mean(est.W_[i])
    
    i+=1

summ = 0
for i in ed : 
    summ += ed[i]
    
for i in ed : 
    ed[i] = ed[i]/summ
    
ed

{'Al': 0.0029460944862789835,
 'C': 0.17774275455069435,
 'Ca': 1.5702379426887525e-28,
 'Cl': 0.00019547956705078234,
 'Cr': 0.004396816659654172,
 'Cu': 0.05399148197227327,
 'Fe': 0.09859615889795532,
 'Ga': 0.016196776005005457,
 'K': 6.92146320514266e-19,
 'Mg': 0.05905013526995491,
 'Mn': 0.0068309549103871955,
 'N': 0.017843994069361376,
 'Na': 0.008969911049002681,
 'Ni': 0.027078075298806736,
 'O': 0.4239864470379724,
 'P': 0.0004205260523634589,
 'Pt': 0.014748308545061532,
 'S': 0.04249751115839008,
 'Si': 0.044000818664340836,
 'Ti': 2.843053756152386e-16,
 'Zn': 0.000507755805446132}

In [39]:
input_filename = str("phyllo_spectre_moyen.hspy")
spectrum = hs.load(input_filename)

spectrum.axes_manager[0].offset += +0.000

spectrum.set_signal_type("EDS_ESMPY")
spectrum.set_analysis_parameters(beam_energy = 300,
azimuth_angle = 0.0,
elevation_angle = 18.0,
elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti', 'Zr'],
tilt_stage = 0.001,
thickness = 80e-7,
density = 2.1,

detector_type = "Det_eff.txt",
width_slope = 0.01,
width_intercept = 0.065,
xray_db = "300keV_corentin_xrays.json")
# fW = spim.set_fixed_W({"p0" : {"Si" : 0.0},"p1" : {"Fe" : 0.0}, "p2" : {"Mg" : 0.0}})

G = spectrum.build_G("bremsstrahlung",norm = True)
est = SmoothNMF( n_components = 1,tol=0.000001, max_iter = 1000, G = G)

est.fit_transform(spectrum.data[:,np.newaxis])
fW = est.W_
fG = est.G_

It 10 / 1000: loss 0.029,  21.254 it/s
It 20 / 1000: loss 0.029,  21.138 it/s
It 30 / 1000: loss 0.029,  21.148 it/s
It 40 / 1000: loss 0.029,  21.006 it/s
It 50 / 1000: loss 0.029,  20.999 it/s
It 60 / 1000: loss 0.029,  20.994 it/s
It 70 / 1000: loss 0.029,  20.939 it/s
It 80 / 1000: loss 0.029,  20.928 it/s
It 90 / 1000: loss 0.029,  20.960 it/s
exits because of relative change < tol: 1.7303514369649291e-06
Stopped after 92 iterations in 0.0 minutes and 4.0 seconds.


In [7]:
x = np.linspace(spectrum.axes_manager[0].offset, spectrum.axes_manager[0].offset + spectrum.axes_manager[0].size*spectrum.axes_manager[0].scale,num = spectrum.axes_manager[0].size)
plt.plot(x,spectrum.data,label="exp")
plt.plot(x,fG@fW,label="fit")
plt.legend()

In [20]:
print_concentrations_from_W(fW[:-2,:],elements = spectrum.metadata.Sample.elements)

Concentrations report
     p0
Al : 0.0177 
C  : 0.0434 
Ca : 0.0000 
Cl : 0.0007 
Cr : 0.0000 
Cu : 0.0189 
F  : 0.0363 
Fe : 0.0866 
Ga : 0.0044 
K  : 0.0005 
Mg : 0.1844 
Mn : 0.0035 
N  : 0.0070 
Na : 0.0087 
Ni : 0.0019 
O  : 0.3663 
P  : 0.0000 
Pt : 0.0013 
S  : 0.0080 
Si : 0.2096 
Ti : 0.0000 
Zr : 0.0008 


In [158]:
k_names = ["Ag_La","Al_Ka", "Ar_Ka","B_Ka","Bi_Ma","C_Ka","Ca_Ka","Ca_La","Cl_Ka","Cr_Ka",
        "Cr_La","Cu_Ka","Cu_La","F_Ka","Fe_Ka","Fe_La","Fe_Lb3","Fe_Ll","Fe_Ln","Ga_Ka",
        "Ga_La","Ho_La","La_La","K_Ka","Mg_Ka","Mn_Ka","Mn_La","Mg_La","N_Ka","Na_Ka",
        "Ni_Ka","Ni_La","O_Ka","P_Ka","Pb_La","Pt_La","Pt_Ma","S_Ka","Si_Ka","Sn_La",
        "Ti_Ka","Tl_Ma","Zr_Ka","Zr_La","Zn_Ka"]
kfactors = [1,
 1.05,
 1.188,
 5,
 1,
 1.42,
 1.16,
 8.8,
 1.093,
 1.314,
 5.23,
 1.773,
 1.523,
 1,
 1.5,
 2.55,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1.119,
 0.94,
 1.5,
 3.36,
 3.36,
 1.18,
 0.98,
 1.587,
 1.582,
 0.95,
 1.096,
 2.85,
 1,
 1,
 1.18,
 1,
 1,
 1.25,
 1,
 3.65,
 1.49,
 1.96]


In [42]:
table, mdata = tu.load_table("300keV_xrays.json")
n_table, n_mdata = tu.import_k_factors(table,mdata, k_names, kfactors,  ref_name = "Si_Ka")
tu.save_table("300keV_corentin_xrays.json",n_table,n_mdata)

In [160]:
didi = {}
for i,j in enumerate(k_names) : 
    didi[j] = kfactors[i]

didi

{'Ag_La': 1,
 'Al_Ka': 1.05,
 'Ar_Ka': 1.188,
 'B_Ka': 5,
 'Bi_Ma': 1,
 'C_Ka': 1.42,
 'Ca_Ka': 1.16,
 'Ca_La': 8.8,
 'Cl_Ka': 1.093,
 'Cr_Ka': 1.314,
 'Cr_La': 5.23,
 'Cu_Ka': 1.773,
 'Cu_La': 1.523,
 'F_Ka': 1,
 'Fe_Ka': 1.5,
 'Fe_La': 2.55,
 'Fe_Lb3': 0,
 'Fe_Ll': 0,
 'Fe_Ln': 0,
 'Ga_Ka': 1,
 'Ga_La': 1,
 'Ho_La': 1,
 'La_La': 1,
 'K_Ka': 1.119,
 'Mg_Ka': 0.94,
 'Mn_Ka': 1.5,
 'Mn_La': 3.36,
 'Mg_La': 3.36,
 'N_Ka': 1.18,
 'Na_Ka': 0.98,
 'Ni_Ka': 1.587,
 'Ni_La': 1.582,
 'O_Ka': 0.95,
 'P_Ka': 1.096,
 'Pb_La': 2.85,
 'Pt_La': 1,
 'Pt_Ma': 1,
 'S_Ka': 1.18,
 'Si_Ka': 1,
 'Sn_La': 1,
 'Ti_Ka': 1.25,
 'Tl_Ma': 1,
 'Zr_Ka': 3.65,
 'Zr_La': 1.49,
 'Zn_Ka': 1.96}

In [166]:
a,b = tu.load_table("300keV_corentin_xrays.json")
a['26']["L3M4"]['cs']/a['26']["KL3"]['cs']
a["26"]

{'KL2': {'energy': 6.3909, 'cs': 5.394061758710123e-23},
 'KL3': {'energy': 6.4039, 'cs': 5.394061758710123e-23},
 'KM2': {'energy': 7.058, 'cs': 3.694394758457301e-24},
 'KM3': {'energy': 7.058, 'cs': 7.277624178144575e-24},
 'L1M2': {'energy': 0.7921, 'cs': 9.729740556306153e-25},
 'L1M3': {'energy': 0.7921, 'cs': 0.0},
 'L2M1': {'energy': 0.6282, 'cs': 0.0},
 'L2M4': {'energy': 0.7175, 'cs': 1.9026533358501573e-23},
 'L3M1': {'energy': 0.6152, 'cs': 0.0},
 'L3M4': {'energy': 0.7045, 'cs': 9.169904989807208e-23},
 'L3M5': {'energy': 0.7045, 'cs': 9.169904989807208e-23}}

In [167]:
cs_sum1 = 0
cs_sum2 = 0
for i in a["26"] : 
    if i[0] == "K" : 
        cs_sum1 += a["26"][i]['cs']
    if i[0] == "L" : 
        cs_sum2 += a["26"][i]['cs']

In [170]:
cs_sum2/cs_sum1

1.7113339363906108

In [169]:
cs_sum1

1.1885325411080435e-22